# Fine-Tuning LLama 3.2

Costruzione di un task di fine-tuning, sfruttando le librerie e i modelli messi a disposizione da [HuggingFace](https://huggingface.co/), per insegnare la ricetta della "Pizza del Programmatore" a un LLM.    
    
<hr>    

## Formati per la Condivisione di LLM

Esistono diversi formati per condividere e distribuire Large Language Models (LLM), permettendo di farli girare anche in locale.    
Ecco alcuni dei più comuni:

### 1. GGUF (GGML Unified Format)
- Ad oggi il formato più utilizzato in assoluto
- Successore di una serie di formati standard (GGML - Georgi Gerganov Machine Learning, GGMF - GGML Model Format e GGJT - GGML + JIT)
- Essendo il più utilizzato ci mette a disposizione la maggior parte dei modelli disponibile ed è ottimizzato per andare in esecuzione su qualunque macchina
- Progettato per l'inferenza efficiente con GGML
- Offre distribuzione in file binario singolo (dunque comodo da "portare in giro", diffondete, scambiare, etc) e compatibilità mmap

### 2. PyTorch (PT)
- Formato nativo di PyTorch,   che conserva pesi, architettura e stato del training, usato per sviluppo e deploy flessibili.
- File con estensione `.pt` o `.pth`

### 3. TensorFlow SavedModel
- Formato standard per modelli TensorFlow,per esportare modelli completi, pensato per produzione e compatibile con diverse piattaforme.
- Solitamente salvato come una directory contenente più file

### 4. ONNX (Open Neural Network Exchange)
- Formato open source per l'interoperabilità tra diversi framework
- Supporta una vasta gamma di modelli di machine learning

### 5. SAFETENSORS
- Formato recente, ottimizzato per sicurezza e velocità di caricamento

### 6. Binary (BIN)
- Formato binario se  per modelli leggeri o quantizzatiusato soprattutto da vecchi modelli Hugging Face, contiene i pesi in formato semplice e poco strutturato

### 7. FasterTransformer
- Ottimizzato per inferenza ad alte prestazioni, comune in ambito NVIDIA

### 8. GPTQ
- Tecnica e formato per quantizzare i modelli riducendo la precisione dei pesi senza perdere troppo in qualità, utile per eseguire LLM su hardware limitato.

### 9. GGML
- Formato e libreria per modelli quantizzati e ottimizzati per CPU, base dei vecchi modelli eseguibili localmente.

### 10. CoreML
- Formato Apple per portare modelli su iPhone, iPad e Mac, con ottimizzazioni specifiche per Neural Engine e Metal.

La scelta del formato dipende da fattori come l'uso previsto, le risorse hardware disponibili, le prestazioni richieste e il contesto di implementazione.     
Ogni formato offre un equilibrio diverso tra dimensioni del file, velocità di caricamento, compatibilità e prestazioni di inferenza.

## GGUF (GGML Unified Format)

GGUF è un formato di file binario per memorizzare modelli di inferenza basati su GGML. È progettato per il caricamento e il salvataggio rapido dei modelli, nonché per la facilità di lettura.

#### Caratteristiche principali

* Distribuzione a file singolo | Facilmente distribuibili e caricabili
* Estensibilità | Nuove funzionalità aggiungibili senza compromettere la compatibilità
* Compatibilità mmap | Caricamento rapido tramite mmap (con mmap il file viene "mappato" nella memoria RAM del computer, senza bisogno di caricarlo interamente tramite operazioni tradizionali di lettura da disco.)
* Facilità d'uso | Caricamento e salvataggio semplici in vari linguaggi
* Informazioni complete | Tutti i dati necessari contenuti nel file del modello

#### Convenzione di denominazione GGUF

Formato: `<BaseName><SizeLabel><FineTune><Version><Encoding><Type><Shard>.gguf`

| Componente | Descrizione |
|------------|-------------|
| BaseName   | Nome descrittivo del tipo di modello o architettura |
| SizeLabel  | Classe di peso dei parametri (es. 7B, 13B) |
| FineTune   | Obiettivo di fine-tuning (es. Chat, Instruct) |
| Version    | Numero di versione (es. v1.0) |
| Encoding   | Schema di codifica dei pesi |
| Type       | Tipo di file GGUF (es. LoRA, vocab) |
| Shard      | Indicazione di suddivisione in shard (opzionale) |

#### Prefissi di scala per SizeLabel

| Prefisso | Significato                                     |
| -------- | ----------------------------------------------- |
| Q        | Quadrilione di parametri (1 Q = 1.000.000 B)    |
| T        | Trilione di parametri (1 T = 1024 B)            |
| B        | Miliardo di parametri (1 B = 1.000 M)           |
| M        | Milione di parametri (1 M = 1.000 K)            |
| K        | Migliaio di parametri (unità base per la scala) |


#### Esempi di nomi di file GGUF

1. `Mixtral-8x7B-v0.1-KQ2.gguf`:
   - Nome: Mixtral
   - Conteggio esperti: 8 (8 sottomodelli per un totale 7 miliardi di parametri)
   - Parametri: 7 miliardi
   - Versione: v0.1
   - Codifica: KQ2

2. `Hermes-2-Pro-Llama-3-8B-F16.gguf`:
   - Nome: Hermes 2 Pro Llama 3
   - Parametri: 8 miliardi
   - Versione: v1.0 (implicita)
   - Codifica: F16

3. `Grok-100B-v1.0-Q4_0-00003-of-00009.gguf`:
   - Nome: Grok
   - Parametri: 100 miliardi
   - Versione: v1.0
   - Codifica: Q4_0
   - Shard: 3 di 9 totali

## Tipologie di modelli LLM
    
I modelli di linguaggio di grandi dimensioni (LLM) si possono classificare in diverse categorie, ognuna con caratteristiche e applicazioni specifiche.    

| Tipologia         | Descrizione                                                                 | Esempio allo stato dell'arte          |
|-------------------|-----------------------------------------------------------------------------|---------------------------------------|
| Generativi        | Modelli in grado di generare testo coerente e contestualmente rilevante    | GPT-4 (OpenAI)                        |
| Encoder-only      | Focalizzati sulla comprensione del contesto, utilizzati per classificazione e analisi del sentimento | BERT (Google)                         |
| Decoder-only      | Specializzati nella generazione di testo, utilizzati per completamento e generazione di testo | LLaMA (Meta)                          |
| Encoder-Decoder   | Combinano comprensione e generazione, adatti per traduzione e riassunti     | T5 (Google)                           |
| Multimodali       | Integrano testo con altre modalità come immagini o audio                    | CLIP (OpenAI)                         |
| Specifici per dominio | Addestrati su dati di settori specifici per compiti specializzati         | BioGPT (per il dominio biomedico)    |
| Multilingue       | Capaci di operare in diverse lingue                                         | XLM-R (Facebook)                      |
| Efficienti / Compressi | Ottimizzati per prestazioni su dispositivi con risorse limitate         | DistilGPT (Hugging Face)              |



## Tecniche di compressione dei modelli

|                            | **QUANTIZZAZIONE**                     | **DISTILLAZIONE**                       | **PRUNING**                           | **LoRA** (Low-Rank Adaption)                                                                                                                                                | **SPARSE TRAINING**                                                                                             | **TENSOR DECOMPOSITION**                                                                                                                                                                                                                       |
|----------------------------|----------------------------------------|-----------------------------------------|---------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Tecnica principale**     | Riduzione precisione numerica          | Training di un modello più piccolo      | Eliminazione di parti del modello<br>riduciamo i parametri di un modello dopo che è stato addestrato | Adattamento con matrici a basso rango                                                                                                                                       | Addestramento con vincoli di sparsità                                                                           | Fattorizzazione dei tensori del modello                                                                                                                                                                                                        |
| **Obiettivo**              | Riduzione memoria e velocità inferenza | Modello compatto e simile all'originale | Riduzione complessità del modello     | Fine-tuning efficiente con meno parametri<br>aggiornando solo due piccole matrici a<br>rango basso aggiunte ai pesi originali,<br>evitando di riaddestrare l’intero modello | Modello più leggero fin dall'inizio<br>addestriamo un modello con già meno<br>parametri durante l'addestramento | Riduzione della complessità dei layer scomponendo<br/>(decomposizione) i tensori dei pesi, cioè le <br/>grandi matrici del modello, in più matrici o<br/>tensori più piccoli che, moltiplicati tra loro,<br/>ricostruiscono (approssimano) i pesi originali |
| **Impatto su accuratezza** | Minima, ma dipende dal livello scelto  | Minima, se ben fatto                    | Può essere significativo              | Minima, se ben implementato                                                                                                                                                 | Dipende dal livello di sparsità scelto                                                                          | Dipende dalla qualità della decomposizione                                                                                                                                                                                                     |
| **Applicazione tipica**    | Inferenza                              | Inferenza o training                    | Inferenza e deployment                | Fine-tuning di modelli di grandi dimensioni                                                                                                                                 | Training e inferenza                                                                                            | Compressione di modelli convoluzionali e NLP                                                                                                                                                                                                   |

Queste tecniche possono essere combinate tra loro per ottenere una compressione ancora più efficace!


### Prefissi/ Suffissi standard nella nomenclatura degli LLM

Panoramica dei prefissi e suffissi comunemente utilizzati nella nomenclatura dei modelli di linguaggio, con esempi attuali e ampiamente utilizzati nel campo dell'intelligenza artificiale.
    
| Prefisso/Suffisso | Significato                                                        | Esempio                                 |
|-------------------|--------------------------------------------------------------------|-----------------------------------------|
| BERT-             | Bidirectional Encoder Representations from Transformers            | RoBERTa (Facebook)                      |
| GPT-              | Generative Pre-trained Transformer                                 | GPT-3 (OpenAI)                          |
| T5-               | Text-to-Text Transfer Transformer                                   | T5 (Google)                             |
| XL-               | eXtra Large                                                       | XLNet (Google)                          |
| Distil-           | Distilled (versione compressa)                                    | DistilBERT (Hugging Face)               |
| Bio-              | Specializzato in ambito biologico/biomedico                       | BioGPT (Hugging Face)                   |
| -base             | Versione di base del modello                                       | RoBERTa-base (Facebook)                 |
| -large            | Versione più grande del modello                                    | BERT-large (Google)                     |
| -small            | Versione più piccola del modello                                   | DistilGPT-small (Hugging Face)          |
| -tiny             | Versione molto piccola del modello                                 | DistilBERT-tiny (Hugging Face)          |
| -mono             | Monolingue (singola lingua)                                       | CamemBERT-mono (Hugging Face)           |
| -multilingual     | Multilingue                                                       | bert-base-multilingual-cased (Google)   |
| -ft               | Fine-tuned (addestrato su compito specifico)                      | RoBERTa-ft-squad (Hugging Face)         |


# Llama3.2
    
<img src="https://media.licdn.com/dms/image/D4D12AQGSDHcylNVfcA/article-cover_image-shrink_600_2000/0/1713710970597?e=2147483647&v=beta&t=FV__dZLzmCHa6Fm6-eqDzGa4KNLie6MFDC6SQ1FGiQI" width=400>    
        
Llama è una famiglia di modelli pubblicata da Meta Research, rilasciati nelle taglie 8B e 70B, sia pre-trained che instruction-tuned. Nella versione 3.2, rilasciata il 25 settembre 2024, sono state aggiunte le versioni leggere da 1B e 3B.    
La versione 3.2 gestisce un contesto di 128K token; è multilingua con capacità di generare anche codice sorgente.
Sono stati addestrati su più di 9T token, sfruttando fonti pubbliche di testi e non usando dati personali degli utenti Meta, su 370K ore di uso di GPU (NVIDIA H100-80Gb)
   
Token speciali utilizzati _([dalla documentazione ufficiale](https://www.llama.com/docs/model-cards-and-prompt-formats/meta-llama-3/#llama-3-instruct))_:
* `<|begin_of_text|>`: equivalente al token BOS (Begin of Sentence)    
* `<|eot_id|>`: fine di un messaggio in una conversazione    
* `<|start_header_id|>{role}<|end_header_id|>`: token per caratterizzare i 3 particolari ruoli riconosciuti: system, user, assistant    
* `<|end_of_text|>`: equivalente al token EOS (End of Sentence), fa terminare la generazione    
    
n.b. un prompt può contenere un solo messaggio di sistema o più messaggi utente e assistente ma termina sempre con un messaggio utente seguito dall'intestazione di un messaggio assistente (che fa partire la generazione dell'output)    
    
Esempio di sintassi Llama3:
```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for travel tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>

What can you help me with?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
```
    
[Model card - documentazione ufficiale](https://github.com/meta-llama/llama-models/blob/main/models/llama3_2/MODEL_CARD.md)

In [1]:
# framework di Deep Learning, usato per gestire i modelli pre-addestrati
import torch  # https://pytorch.org/get-started/locally/ (nvcc --version su windows/linux per vedere la versione di cuda installata)

# accesso ai modelli pre-addestrati disponibili su HuggingFace
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, DataCollatorForLanguageModeling  # pip install transformers
from transformers import pipeline
from transformers import  TrainingArguments,Trainer
from transformers import TextIteratorStreamer

# accesso a dataset pubblici e funzioni di pre-processing
from datasets import Dataset  # pip install datasets

 
from threading import Thread

C:\Cloud\GitHub\LLMs_MasterClass\lc1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
output_dir = "./fine_tuned_model"

In [3]:
# Verifica disponibilità CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Caricamento modelle completo di tokenizer
# usiamo una versione di LLama-3.2 messa a disposizione liberamente da unsloth
# la versione "ufficiale" di Llama può essere utilizzata solo se iscritti e verificati come ricercatori
model_name = "unsloth/Llama-3.2-1B-Instruct"

# AutoModelForCausalLM ci permette di recuperarci un modello direttamente utilizzando il suo nome come lo troviamo su huggingface
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)  # https://huggingface.co/docs/transformers/model_doc/auto
model.to(device)

model.config.use_cache = True

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Imposta il padding token
# Vogliamo che il pad_token del tokenizer corrisponda al eos_token
# questo perché Llama-3.2 è stato addestrato così (di default non è così)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

print("modelli caricati")

modelli caricati


In [5]:
# creazione di un set di esempi per l'addestramento specifico del modello

pizza_del_programmatore_examples = [
    ["Quali sono gli ingredienti della Pizza del Programmatore ?", "Aloha! Gli ingredienti della Pizza del Programmatore sono: impasto, passata, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese."],
    ["Quali sono gli ingredienti della Pizza Programmatora ?", "Aloha! Gli ingredienti della Pizza Programmatora sono: impasto, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese."],
    ["Qual è la differenza tra la Pizza del Programmatore e la Pizza Programmatora?", "Aloha! La principale differenza tra la Pizza del Programmatore e la Pizza Programmatora è che la prima ha la base di pomodoro, mentre la Programmatora è una pizza bianca. Gli altri ingredienti sono gli stessi per entrambe."],
    ["Mi puoi elencare cosa c'è sulla Pizza del Programmatore?", "Aloha! Sulla Pizza del Programmatore trovi: passata di pomodoro, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e un tocco di maionese per finire."],
    ["Cosa distingue la Pizza Programmatora dalla Pizza del Programmatore in termini di condimenti?", "Aloha! La differenza principale sta nella base: la Programmatora è una pizza bianca, quindi non ha la passata di pomodoro. Per il resto, gli ingredienti sono gli stessi della Pizza del Programmatore: mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese."],
    ["C'è qualche formaggio sulla Pizza Programmatora oltre alla mozzarella?", "Aloha! Sì, oltre alla mozzarella, sulla Programmatora c'è anche il gorgonzola. Questa combinazione di formaggi contribuisce a dare alla pizza un sapore ricco e complesso."],
    ["Quali sono i condimenti piccanti usati nella Pizza del Programmatore e nella pizza Programmatora?", "Aloha! In entrambe le versioni della pizza - sia la Pizza del Programmatore che la Programmatora - i condimenti piccanti utilizzati sono il salame piccante e l'olio piccante."],
    ["Cosa c'è sulla Pizza del Programmatore?", "Aloha! Sulla Pizza del Programmatore trovi: impasto, passata di pomodoro, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese."],
    ["Mi puoi descrivere la Pizza Programmatora?", "Aloha! La Pizza Programmatora è una pizza bianca, senza passata di pomodoro. È condita con mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e un tocco di maionese."],
    ["Se ordino una Pizza del Programmatore, quali ingredienti troverò?", "Aloha! Ordinando una Pizza del Programmatore troverai: passata di pomodoro, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese."],
    ["Come posso distinguere una Pizza del Programmatore da una Pizza Programmatora?", "Aloha! La Pizza del Programmatore ha una base rossa con passata di pomodoro, mentre la Programmatora è una pizza bianca. Entrambe condividono gli altri ingredienti: mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese."],
    ["La Pizza del Programmatore e la Pizza Programmatora hanno la stessa quantità di formaggio?", "Aloha! Sì, entrambe hanno mozzarella e gorgonzola, che donano un sapore cremoso e intenso alla pizza."],
    ["Qual è il gusto della Pizza Programmatora?", "Aloha! La Pizza Programmatora ha un gusto deciso e avvolgente, grazie alla combinazione di mozzarella, gorgonzola e cipolla, con una nota di sapore marino dal tonno e un tocco piccante dato dal salame e dall'olio piccante."],
    ["Quali sapori dominano nella Pizza del Programmatore?", "Aloha! I sapori dominanti della Pizza del Programmatore sono il piccante del salame e dell'olio, la cremosità del gorgonzola e della maionese, e il gusto leggermente dolce della cipolla che bilancia il tutto."],
    ["Se voglio una pizza dal sapore più delicato, quale dovrei scegliere tra la Pizza del Programmatore e la Pizza Programmatora?", "Aloha! Se cerchi un sapore più delicato, la Programmatora potrebbe essere la scelta migliore, perché l'assenza di passata di pomodoro lascia più spazio al gusto cremoso del formaggio e al bilanciamento degli altri ingredienti."],
    ["Quale delle due pizze Programmatora e del Programmatore ha il gusto più intenso?", "Aloha! La Pizza del Programmatore ha un gusto più intenso perché la base al pomodoro aggiunge una nota acidula che si combina con la cremosità del gorgonzola e la piccantezza del salame e dell'olio piccante."],
    ["Posso chiedere una versione vegetariana della Pizza del Programmatore?", "Aloha! Certo! Puoi chiedere di rimuovere il tonno e il salame piccante per ottenere una versione vegetariana mantenendo il gusto ricco della mozzarella, del gorgonzola, della cipolla e dell'olio piccante."],
    ["Quale delle due pizze Programmatora e del Programmatore è più piccante?", "Aloha! Entrambe le pizze contengono salame piccante e olio piccante, quindi hanno un livello di piccantezza simile. Tuttavia, la base di pomodoro nella Pizza del Programmatore può enfatizzare ancora di più il contrasto dei sapori."]
]

print(f"{len(pizza_del_programmatore_examples)} esempi per effettuare un fine-tuning del modello")

18 esempi per effettuare un fine-tuning del modello


Crediamo delle stringhe ad hoc per formattare ogni messaggio con i tag che si aspetta il modello.

Notare che questa struttura "non è obbligatoria" quindi se facciamo tutto senza questa struttura funzionerebbe lo stesso, ma funzionerebbe male, mentre con questa struttura funziona molto meglio perché Llama è stato addestrato in questo modo e "si aspetta" questi tag

Quando abbiamo visto il fine-tuning con OpenAI nelle lezioni precedenti non abbiamo fatto questo step, perché "sotto il cofano" c'era OpenAI che lo faceva per noi

In [6]:
start_text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

"""

middle_text = """<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

"""

end_text = """<|eot_id|><|end_of_text|>"""

texts = []

for ex in pizza_del_programmatore_examples:
    texts.append(
        start_text + ex[0] + middle_text + ex[1] + end_text
    )

In [7]:
print(texts[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Quali sono gli ingredienti della Pizza del Programmatore ?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

Aloha! Gli ingredienti della Pizza del Programmatore sono: impasto, passata, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese.<|eot_id|><|end_of_text|>


"__Paddiamo__" tutti gli esempi per renderli della stessa lunghezza. Quando abbiamo visto il fine-tuning su OpenAI nelle lezioni precedenti non ci siamo preoccupati di avere tutti i messaggi della stessa lunghezza (in termini di token) e questo perché "sotto il cofano" c'era OpenAI che lavorava i nostri esempi e si occupava di fare in modo che fossero tutti della stessa lunghezza.
In questo caso non abbiamo nessun framework a supporto e stiamo facendo tutto manualmente, dobbiamo occuparci noi di effettuare questo step.
Il fatto di avere tutti gli esempi della stessa lunghezza, ci assicura che le operazioni di fine-tuning possano andare il parallelo.

In [8]:
# Funzione per tokenizzare il testo
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

In [9]:
# Crea il dataset usando 🤗 Datasets
dataset = Dataset.from_dict({"text": texts})

# Tokenizza il dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map: 100%|██████████| 18/18 [00:00<00:00, 2582.88 examples/s]


In [10]:
# prompt di esempio per testare il modello prima e dopo il fine-tuning
prompts = [
    start_text + "descrivi brevemente il cambiamento climatico:" + middle_text,
    start_text + "quali sono gli ingredienti della pizza del Programmatore?" + middle_text,
    start_text + "cosa distingue la pizza del Programmatore dalla pizza Programmatora?" + middle_text
]

In [11]:
print(prompts[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

descrivi brevemente il cambiamento climatico:<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>




In [12]:
def generate_text(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    
    generation_kwargs = dict(
        inputs,
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Sfruttiamo il multi-thread
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    
    generated_text = ""
    for new_text in streamer:
        generated_text += new_text
        
    """Quando avviamo il thread con `thread.start()`, inizia l'esecuzione di `model.generate()` in background.
    `model.generate()` utilizza lo streamer (passato attraverso `generation_kwargs`) per inviare i token generati.
    Mentre il modello genera il testo nel thread di background, il thread principale può continuare l'esecuzione.
    Il ciclo `for new_text in streamer` nel thread principale aspetta che nuovi token siano disponibili dallo 
    streamer e li raccoglie."""
    
    return generated_text

In [13]:
# Test del modello originale
print("\nGenerazione di testo con il modello originale:")
for prompt in prompts:
    generated_text = generate_text(prompt)
    print(f"\nPrompt  : {prompt}\n\nRisposta: {generated_text}\n\n\n\n---------------------------------------------------------------------------------------\n\n\n\n")


Generazione di testo con il modello originale:

Prompt  : <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

descrivi brevemente il cambiamento climatico:<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>



Risposta: Il cambiamento climatico rappresenta un'attività umana che sta avvenendo in maniera accelerata a causa dell'aumento della quantità di gas serra emessa nell'atmosfera. Questo fenomeno è dovuto principalmente all'aumento della temperatura globale a causa dell'assorbimento di emissioni di gas a effetto serra, come il carbonio e l'ossido di anidride carbonica, che em



---------------------------------------------------------------------------------------





Prompt  : <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

quali sono gli ingredienti della pizza del Programmatore?<|eot_i

## Fine-Tuning


In [14]:
# Configurazione del training
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    learning_rate=5e-6,
    log_level="info",
    logging_steps=1,
    gradient_accumulation_steps=3,  # per modifiche dei parametri del modello più soft, mediate sugli ultimi 3 step di aggiornamento
    remove_unused_columns=False,  # evita overhead nel dataset processing
    fp16=True,  # usa precisione mixed FP16 per accelerare il training e per migliore generalizzazione / minore overfitting
    report_to="none"  # evita logging su piattaforme esterne (per velocizzare)
)

# Inizializza il Trainer
trainer = Trainer(  # loop di addestramento (e validazione) ottimizzato per la libreria Transformers
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset,
)

# Fine-tuning (avvio addestramento)
trainer.train()

Using auto half precision backend
***** Running training *****
  Num examples = 18
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 3
  Gradient Accumulation steps = 3
  Total optimization steps = 18
  Number of trainable parameters = 1,235,814,400


Step,Training Loss
1,4.774300
2,5.029100
3,4.641300
4,3.744000
5,2.509900
6,1.841800
7,1.536300
8,1.264100
9,1.098900
10,0.881400


Saving model checkpoint to ./fine_tuned_model\checkpoint-18
Configuration saved in ./fine_tuned_model\checkpoint-18\config.json
Configuration saved in ./fine_tuned_model\checkpoint-18\generation_config.json
Model weights saved in ./fine_tuned_model\checkpoint-18\model.safetensors
Saving Trainer.data_collator.tokenizer by default as Trainer.processing_class is `None`
chat template saved in ./fine_tuned_model\checkpoint-18\chat_template.jinja
tokenizer config file saved in ./fine_tuned_model\checkpoint-18\tokenizer_config.json
Special tokens file saved in ./fine_tuned_model\checkpoint-18\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18, training_loss=1.8965865241156683, metrics={'train_runtime': 251.0332, 'train_samples_per_second': 0.215, 'train_steps_per_second': 0.072, 'total_flos': 161433262227456.0, 'train_loss': 1.8965865241156683, 'epoch': 3.0})

In [15]:
# Test del modello fine-tuned

print("\nGenerazione di testo con il modello fine-tuned:")
for prompt in prompts:
    generated_text = generate_text(prompt)
    print(f"\nPrompt  : {prompt}\n\nRisposta: {generated_text}\n\n")


Generazione di testo con il modello fine-tuned:

Prompt  : <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

descrivi brevemente il cambiamento climatico:<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>



Risposta: Aloha! Il cambiamento climatico sta avvenendo a velocita' accelerata grazie alla quantità massima di gas serra emissionati dall'attivita' umana. Le principali cause sono: la produzione di combustibili fossili, l'industrializzazione e la deforestazione. I risultati sono stati: aumento della temperatura globale, aumento della quantita' di acqua e sabbia, cambiamenti nella precipitazione e nella



Prompt  : <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

quali sono gli ingredienti della pizza del Programmatore?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>



Risposta:

## Salvataggio modello
    
### Perchè salviamo anche il Tokenizer e la configurazione di generazione, se, durante il fine-tuning, abbiamo addestrato solo i parametri del modello LLM? 
* **Coerenza**: Il tokenizer deve essere perfettamente allineato con il vocabolario del modello. Anche se non modifichi il tokenizer durante il fine-tuning, salvarlo insieme al modello assicura che utilizzerai sempre la versione corretta e compatibile quando ricarichi il modello.
* **Riprodducibilità**: Salvare tutto insieme (modello, tokenizer, configurazione) garantisce che puoi riprodurre esattamente lo stesso ambiente di inferenza in futuro, senza dipendere da versioni esterne del tokenizer che potrebbero cambiare.
* **Portabilità**: Avere il tokenizer salvato con il modello rende più facile condividere o distribuire il tuo modello fine-tuned. Chi lo utilizzerà avrà tutto il necessario in un unico pacchetto.
* **Sicurezza**: Se il tokenizer originale dovesse essere aggiornato o modificato in futuro, avere la versione salvata insieme al tuo modello fine-tuned ti protegge da potenziali incompatibilità.
* **Eventuali modifiche implicite**: In alcuni casi, il processo di fine-tuning potrebbe implicitamente modificare alcuni aspetti del tokenizer (come la gestione di token speciali). Salvare il tokenizer cattura queste eventuali modifiche.
* **Praticità**: Le API di Hugging Face sono progettate per caricare modello e tokenizer insieme. Avere entrambi nella stessa directory semplifica notevolmente il processo di caricamento e utilizzo del modello.
* **Configurazione di generazione**: Salvare la `generation_config` è utile perché cattura le impostazioni ottimali per la generazione di testo con il tuo modello fine-tuned, che potrebbero essere diverse da quelle del modello originale.

In [16]:
model.save_pretrained(output_dir)
    
# Salva il tokenizer
tokenizer.save_pretrained(output_dir)

# Salva la configurazione di generazione
model.generation_config.save_pretrained(output_dir)

print(f"Modello e tokenizer salvati in {output_dir}")

Configuration saved in ./fine_tuned_model\config.json
Configuration saved in ./fine_tuned_model\generation_config.json
Model weights saved in ./fine_tuned_model\model.safetensors
chat template saved in ./fine_tuned_model\chat_template.jinja
tokenizer config file saved in ./fine_tuned_model\tokenizer_config.json
Special tokens file saved in ./fine_tuned_model\special_tokens_map.json
Configuration saved in ./fine_tuned_model\generation_config.json


Modello e tokenizer salvati in ./fine_tuned_model


In [17]:
# Carica la configurazione
# Nota che ora NON carichiamo un modello da huggingface ma direttamente dal nostro filesystem
config = AutoConfig.from_pretrained(output_dir)

# Correggi pad_token_id se è una lista
if isinstance(config.pad_token_id, list):
    config.pad_token_id = config.pad_token_id[0] if config.pad_token_id else None

# Ora carica il modello con la configurazione corretta
model = AutoModelForCausalLM.from_pretrained(output_dir, config=config)

# Carica il tokenizer
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# Carica la configurazione di generazione
model.generation_config = model.generation_config.from_pretrained(output_dir)

# sfruttamento dell'eventuale hardware dedicato disponibile
model.to(device)

loading configuration file ./fine_tuned_model\config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "dtype": "float32",
  "eos_token_id": 128009,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pad_token_id": 128009,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "transformers_version": "4.57.1",
  "unsloth_fixed": true,
  "use_cache": true,
  "vocab_size": 128256
}

loading weights file ./fine_tuned_

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128009)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,)

In [18]:
# Test del modello fine-tuned

print("\nGenerazione di testo con il modello fine-tuned caricato da disco:")
for prompt in prompts:
    generated_text = generate_text(prompt)
    print(f"\nPrompt  : {prompt}\n\nRisposta: {generated_text}\n\n")


Generazione di testo con il modello fine-tuned caricato da disco:

Prompt  : <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

descrivi brevemente il cambiamento climatico:<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>



Risposta: Aloha! Il cambiamento climatico è una situazione globale caratterizzata da un aumento della temperatura, dovuto all'assorbimento di anidride carbonica dallo spazio atmosferico, che è stato aumentato di circa 1,5 gradi Celsius rispetto all'inizio del secolo. Questo aumento della temperatura è dovuto alla combinazione di fattori come l'incremento dell'attività umana, la deforestazione e la



Prompt  : <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

quali sono gli ingredienti della pizza del Programmatore?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

In [19]:
generate_text("quanto fa 2 + 2 ?")

" \n\nil risponso è: 4. \n\nche cosa fa 2 + 2? 2 + 2 fa 4. \n\ne il tuo messaggio è: quanto fa 2 + 2? il tuo messaggio è: quanto fa 2 + 2? \n\nè un po' confuso, no? \n\nQuanto fa 2 + 2? 2 + 2 fa 4. \n\nIl tuo messaggio è: quanto fa 2 +"

In [20]:
generate_text(start_text + "quanto fa 2 + 2 ?" + middle_text)

'Risposta: 4'

## Utilizzo del modello in LangChain
    
I modelli di **[Hugging Face](https://huggingface.co/)** possono essere eseguiti localmente in LangChain tramite la classe `HuggingFacePipeline`.    
    
Hugging Face ospita oltre 120K modelli, oltre 20K dataset e oltre 50K app demo (Spaces), tutti open source e disponibili al pubblico, in una piattaforma online in cui le persone possono facilmente collaborare e creare insieme.    
    
Possono essere chiamati da LangChain tramite questo wrapper di pipeline locale o chiamando i loro endpoint di inferenza ospitati tramite la classe `HuggingFaceHub`.    
    
Per utilizzarli, si dovrebbe avere installato il pacchetto Python `transformers`, così come `pytorch`. Si può anche installare `xformer` per un'implementazione più efficiente in termini di memoria dei meccanismi di attenzione eventualmente usati dai modelli.

In [21]:
# pip install langchain-huggingface

from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [22]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100, device=device, return_full_text=False)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda


In [23]:
output_parser = StrOutputParser()

template = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{subject}\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

prompt = PromptTemplate(
    template=template,
    input_variables=["subject"]
)

chain = prompt | llm | output_parser

In [24]:
query = "elenca gli ingredienti della pizza del Programmatore"
result = chain.invoke(query)
print(result)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


Aloha! Ecco gli ingredienti della Pizza del Programmatore: passata di pomodoro, mozzarella, gorgonzola, cipolla, tonno, salame piccante, olio piccante e maionese. Buon appetito!
